In [4]:
'''
2048 GAME PROJECT: Analysis of the bot efficiency and accuracy.

Date created:
    06/2022

Author:
    Filip J. Cierkosz
'''

# Analyze the accuracy of the obtaining the goal state.
# Analyze the runtime for winning ones.
# Find the fastest time to win the game.
# Find the slowest time to win the game.
# ...anything else to be added.
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt


In [5]:
db = sqlite3.connect('../db/bot_records.db')
df = pd.read_sql_query("SELECT * FROM bot_records", db)

In [6]:
len(df[df['win']==1])

150

In [7]:
len(df[df['win']==0])

350

In [8]:
df.tail(10)

,id,score,win,time_played_sec,date_played
490,491,1024,0,66.830692,29 Jun 2022 08:14:02 AM
491,492,2048,1,69.057199,29 Jun 2022 08:15:12 AM
492,493,2048,1,68.627941,29 Jun 2022 08:16:21 AM
493,494,1024,0,54.530487,29 Jun 2022 08:17:17 AM
494,495,1024,0,65.225030,29 Jun 2022 08:18:23 AM
495,496,2048,1,69.839370,29 Jun 2022 08:19:33 AM
496,497,1024,0,68.122755,29 Jun 2022 08:20:43 AM
497,498,1024,0,53.485373,29 Jun 2022 08:21:37 AM
498,499,1024,0,52.089843,29 Jun 2022 08:22:30 AM
499,500,2048,1,66.728309,29 Jun 2022 08:23:38 AM


In [9]:
len(df[df['score']>=1024])

440

In [10]:
df[df['score']>=1024]

,id,score,win,time_played_sec,date_played
0,1,1024,0,68.574313,28 Jun 2022 11:45:28 PM
1,2,2048,1,69.399666,28 Jun 2022 11:46:38 PM
3,4,2048,1,67.006177,28 Jun 2022 11:48:23 PM
4,5,1024,0,67.183569,28 Jun 2022 11:49:31 PM
6,7,1024,0,65.865621,28 Jun 2022 11:51:15 PM
...,...,...,...,...,...
495,496,2048,1,69.839370,29 Jun 2022 08:19:33 AM
496,497,1024,0,68.122755,29 Jun 2022 08:20:43 AM
497,498,1024,0,53.485373,29 Jun 2022 08:21:37 AM
498,499,1024,0,52.089843,29 Jun 2022 08:22:30 AM


In [11]:
df['time_played_sec'].mean()

61.41298767614364